In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from tqdm.notebook import tqdm
import time
import random
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/2023/ML-Competitions

/content/drive/MyDrive/2023/ML-Competitions


# Get Data and Split

In [ ]:
train_data = pd.read_csv('./data/train.csv')
train_data.head()

,traffic(t-10),traffic(t-9),traffic(t-8),traffic(t-7),traffic(t-6),traffic(t-5),traffic(t-4),traffic(t-3),traffic(t-2),traffic(t-1),traffic(t),type
0,3.0,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,0
1,2.0,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,0
2,5.0,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,0
3,6.0,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,6.0,0
4,6.0,4.0,6.0,6.0,3.0,12.0,6.0,6.0,4.0,6.0,17.0,0


In [ ]:
train_data.shape

(12645, 12)

In [ ]:
n_train = train_data.shape[0]
n_real_train = int(n_train*0.8)
n_val = n_train-n_real_train
print(f"num of train: {n_real_train} val: {n_val}")

num of train: 10116 val: 2529


In [ ]:
X,y = train_data.drop('type',axis = 1).to_numpy(),train_data['type'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from collections import Counter
Counter(y_train)

Counter({0: 6290, 1: 2370, 3: 736, 2: 720})

In [ ]:
Counter(y_val)

Counter({0: 1596, 1: 573, 2: 188, 3: 172})

# Stadardize

In [ ]:
scaled_train_X = (X_train-X_train.mean())/X_train.std()
scaled_val_X = (X_val-X_train.mean())/X_train.std()

In [ ]:
global best_model_dict
best_model_dict = dict()

# Logistic Regression (multi-class)

ref: http://deeplearning.stanford.edu/tutorial/supervised/SoftmaxRegression/

In [ ]:
from sklearn.utils import shuffle
class Multiclass:
    def __init__(self, max_iter, eta, mu):
      self.max_iter = max_iter
      self.eta = eta
      self.mu = mu

    def fit(self, X, Y):
        Y_onehot = onehot_encoder.fit_transform(Y.reshape(-1,1))
        limit = np.sqrt(1/X.shape[1])
        self.W = np.random.uniform(-limit,limit,(X.shape[1], Y_onehot.shape[1]))
        n_epoch = self.max_iter
        use_X,use_Y = X.copy(),Y.copy()
        for epoch in tqdm(range(n_epoch)):
          X, Y= shuffle(use_X,use_Y, random_state=epoch)
          self.loss_steps, self.W = gradient_descent(X, Y,self.max_iter,self.eta,self.mu,self.W)
          pred_X = self.predict(X)
          train_acc = np.sum(pred_X == Y)/len(Y)
          print(f"epoch:{epoch} acc:{train_acc}")

    def loss_plot(self):
        return self.loss_steps.plot(
            x='step', 
            y='loss',
            xlabel='step',
            ylabel='loss'
        )

    def predict(self, H):
        Z = - H @ self.W
        P = softmax(Z, axis=1)
        return np.argmax(P, axis=1)

In [ ]:
X = scaled_train_X
Y = train_y

In [ ]:
import warnings
warnings.filterwarnings("ignore")

eta_ = [0.1,0.01]
mu_ = [0.01,0.05,0.1,1]
model = Multiclass(max_iter=1,eta=0.1,mu=1)
model.fit(X, Y)

  0%|          | 0/1 [00:00<?, ?it/s]

epoch:0 acc:0.7902332937920127


# SVM

In [ ]:
import numpy as np
from sklearn.utils import shuffle
class SVM:
    def __init__(self, C=0.1, lr=0.001, max_iter=5000, tol=1e-4):
        self.C = C  # regularization parameter
        self.lr = lr  # learning rate
        self.max_iter = max_iter  # maximum number of iterations
        self.tol = tol  # tolerance for stopping criterion
        self.W = None  # weights
        self.b = None  # bias

    def fit(self, X_, y_):
        # convert labels to one-hot encoding
        X,y = X_.copy(),y_.copy()
        y = np.eye(len(np.unique(y)))[y]
        print("one-hot y:",y.shape)
        # initialize weights and bias
        self.W = np.zeros((X.shape[1], y.shape[1]))
        self.b = np.zeros(y.shape[1])

        # gradient descent
        for i in range(self.max_iter):
            # compute margins
            shuffled_X, shuffled_y= shuffle(X,y, random_state=i)
            margin = shuffled_y * (shuffled_X.dot(self.W) + self.b)
            # compute hinge loss
            loss = np.maximum(0, 1 - margin)
            # compute regularized loss
            reg_loss = 0.5 * np.sum(self.W ** 2) + self.C * np.sum(loss)
            # compute gradients
            dW = - shuffled_X.T.dot(shuffled_y * (loss > 0)) + self.C * self.W
            db = - np.sum(shuffled_y * (loss > 0), axis=0)
            # update weights and bias
            self.W -= self.lr * dW
            self.b -= self.lr * db
            # check stopping criterion
            if np.linalg.norm(dW) < self.tol:
                break

    def predict(self, X):
        # compute margins
        margin = X.dot(self.W) + self.b
        # compute class scores
        scores = np.argmax(margin, axis=1)
        return scores


In [ ]:
import copy
lrs = [0.1,0.01,0.005,0.001,0.0001]
C = [0.001,0.01,0.1,1.,5.]
best_param = dict()

best_acc = 0
for C_ in C:
  for lr in lrs:
    print(f"\n C:{C_} lr:{lr}")
    svm = SVM(lr =lr,C =C_)
    svm.fit(scaled_train_X, y_train)
    # predict labels
    y_pred = svm.predict(scaled_train_X)
    print("predictions:",Counter(y_pred))
    # compute accuracy
    acc = np.mean(y_pred == y_train)
    print('Accuracy:', acc)
    if acc>best_acc:
      best_acc = acc
      best_svm = copy.deepcopy(svm)
      best_param['C']=C_
      best_param['lr']=lr


 C:0.001 lr:0.1
one-hot y: (10116, 4)
predictions: Counter({0: 8031, 1: 2085})
Accuracy: 0.7837089758797944

 C:0.001 lr:0.01
one-hot y: (10116, 4)
predictions: Counter({0: 8020, 1: 2096})
Accuracy: 0.7845986555950969

 C:0.001 lr:0.005
one-hot y: (10116, 4)
predictions: Counter({0: 8012, 1: 2104})
Accuracy: 0.7850929221035983

 C:0.001 lr:0.001
one-hot y: (10116, 4)
predictions: Counter({0: 7962, 1: 2154})
Accuracy: 0.7890470541716094

 C:0.001 lr:0.0001
one-hot y: (10116, 4)
predictions: Counter({3: 7758, 1: 2358})
Accuracy: 0.2539541320680111

 C:0.01 lr:0.1
one-hot y: (10116, 4)


KeyboardInterrupt: ignored

In [ ]:
best_param

# One-vs-All (Imbalance 때문에)

## Best models storage

In [ ]:
global best_svm_dict
best_svm_dict = dict()

In [ ]:
import numpy as np

class BinarySVM:
    def __init__(self, C=1, learning_rate=0.01, num_iterations=1000):
        self.C = C
        self.lr = learning_rate
        self.num_iterations = num_iterations
        self.W = None
        self.b = None
        
    def fit(self, X, y):
        y = np.where(y == 0, -1, 1)

        # Initialize weights and bias
        num_features = X.shape[1]
        self.W = np.zeros(num_features)
        self.b = 0
        for i in range(self.num_iterations):
            scores = np.dot(X, self.W) + self.b
            margin = np.multiply(y, scores)
            hinge_loss = np.maximum(0, 1 - margin)
            loss = np.mean(hinge_loss) + 0.5 * self.C * np.sum(self.W ** 2)

            # Compute gradients
            binary = np.zeros(len(margin))
            binary[hinge_loss > 0] = 1
            binary_margin = np.multiply(binary, y)
            dW = -np.dot(binary_margin, X) + self.C * self.W
            db = -np.sum(binary_margin)

            # Update weights and bias
            self.W -= self.lr * dW
            self.b -= self.lr * db

    def predict(self, X):
        scores = np.dot(X, self.W) + self.b
        y_pred = np.where(scores >= 0, 1, 0)
        return y_pred

    def predict_proba(self, X):
        scores = np.dot(X, self.W) + self.b
        probs = 1 / (1 + np.exp(-scores))
        probs = np.vstack((1 - probs, probs)).T
        return probs


In [ ]:
y_train0 = (y_train == 0)*1
y_train1 = (y_train == 1)*1
y_train2 = (y_train == 2)*1
y_train3 = (y_train == 3)*1

y_val0 = (y_val==0)*1
y_val1 = (y_val==1)*1
y_val2 = (y_val==2)*1
y_val3 = (y_val==3)*1

In [ ]:
print(Counter(y_train0))
print(Counter(y_train1))
print(Counter(y_train2))
print(Counter(y_train3))

Counter({1: 6290, 0: 3826})
Counter({0: 7746, 1: 2370})
Counter({0: 9396, 1: 720})
Counter({0: 9380, 1: 736})


In [ ]:
import copy
lrs = [0.1,0.01,0.005,0.001,0.0005,0.0001]
C = [0.001,0.01,0.1,1.,5.]
best_param0 = dict()

best_acc_0 = 0
for C_ in C:
  for lr in lrs:
    svm0 = BinarySVM(C=C_,learning_rate=lr)
    svm0.fit(scaled_train_X, y_train0)
    y_pred0 = svm0.predict(scaled_train_X)
    y_val_pred0 = svm0.predict(scaled_val_X)
    acc0 = np.mean(y_pred0 == y_train0)
    val_acc0 = np.mean(y_val_pred0==y_val0)
    print(f"(lr:{lr}, C:{C_} label:0) acc: {round(acc0,4)} val acc:{round(val_acc0,3)}")
    if val_acc0>best_acc_0:
      best_acc_0 = val_acc0
      best_param0['lr'] = lr
      best_param0['C']=C_
      best_svm0 = copy.deepcopy(svm0)

(lr:0.1, C:0.001 label:0) acc: 0.9014 val acc:0.907
(lr:0.01, C:0.001 label:0) acc: 0.9016 val acc:0.907
(lr:0.005, C:0.001 label:0) acc: 0.9091 val acc:0.912
(lr:0.001, C:0.001 label:0) acc: 0.9134 val acc:0.919
(lr:0.0005, C:0.001 label:0) acc: 0.9059 val acc:0.91
(lr:0.0001, C:0.001 label:0) acc: 0.9145 val acc:0.922
(lr:0.1, C:0.01 label:0) acc: 0.9062 val acc:0.911
(lr:0.01, C:0.01 label:0) acc: 0.9057 val acc:0.91
(lr:0.005, C:0.01 label:0) acc: 0.3782 val acc:0.369
(lr:0.001, C:0.01 label:0) acc: 0.9162 val acc:0.921
(lr:0.0005, C:0.01 label:0) acc: 0.9083 val acc:0.912
(lr:0.0001, C:0.01 label:0) acc: 0.9134 val acc:0.919
(lr:0.1, C:0.1 label:0) acc: 0.9015 val acc:0.907
(lr:0.01, C:0.1 label:0) acc: 0.915 val acc:0.923
(lr:0.005, C:0.1 label:0) acc: 0.9121 val acc:0.916
(lr:0.001, C:0.1 label:0) acc: 0.9027 val acc:0.908
(lr:0.0005, C:0.1 label:0) acc: 0.9143 val acc:0.921
(lr:0.0001, C:0.1 label:0) acc: 0.9115 val acc:0.916
(lr:0.1, C:1.0 label:0) acc: 0.8483 val acc:0.851
(l

In [ ]:
print(best_param0)
print(best_acc_0)

{'lr': 0.01, 'C': 0.1}
0.9228944246737841


In [ ]:
import copy
lrs = [0.0009,0.001,0.0015,0.002]
C = [0.0099,0.01,0.011,0.015]

for C_ in C:
  for lr in lrs:
    svm0 = BinarySVM(C=C_,learning_rate=lr)
    svm0.fit(scaled_train_X, y_train0)
    y_pred0 = svm0.predict(scaled_train_X)
    y_val_pred0 = svm0.predict(scaled_val_X)
    acc0 = np.mean(y_pred0 == y_train0)
    val_acc0 = np.mean(y_val_pred0==y_val0)
    print(f"(lr:{lr}, C:{C_} label:0) acc: {round(acc0,4)} val acc:{round(val_acc0,3)}")
    if val_acc0>best_acc_0:
      best_acc_0 = val_acc0
      best_param0['lr'] = lr
      best_param0['C']=C_

(lr:0.0009, C:0.0099 label:0) acc: 0.9135 val acc:0.919
(lr:0.001, C:0.0099 label:0) acc: 0.9138 val acc:0.92
(lr:0.0015, C:0.0099 label:0) acc: 0.9029 val acc:0.909
(lr:0.002, C:0.0099 label:0) acc: 0.9095 val acc:0.913
(lr:0.0009, C:0.01 label:0) acc: 0.9102 val acc:0.915
(lr:0.001, C:0.01 label:0) acc: 0.9162 val acc:0.921
(lr:0.0015, C:0.01 label:0) acc: 0.7444 val acc:0.752
(lr:0.002, C:0.01 label:0) acc: 0.3782 val acc:0.369
(lr:0.0009, C:0.011 label:0) acc: 0.9027 val acc:0.909
(lr:0.001, C:0.011 label:0) acc: 0.9067 val acc:0.911
(lr:0.0015, C:0.011 label:0) acc: 0.9191 val acc:0.925
(lr:0.002, C:0.011 label:0) acc: 0.91 val acc:0.914
(lr:0.0009, C:0.015 label:0) acc: 0.8885 val acc:0.897
(lr:0.001, C:0.015 label:0) acc: 0.9027 val acc:0.908
(lr:0.0015, C:0.015 label:0) acc: 0.9191 val acc:0.925
(lr:0.002, C:0.015 label:0) acc: 0.9023 val acc:0.908


In [ ]:
print(best_param0)
print(best_acc_0)

{'lr': 0.0015, 'C': 0.011}
0.9252669039145908


In [ ]:
import copy
lrs = [0.0014,0.0015,0.0016]
C = [0.01,0.011,0.015,0.0112]
# best_param0 = dict()
# best_acc_0 = 0

for C_ in C:
  for lr in lrs:
    svm0 = BinarySVM(C=C_,learning_rate=lr)
    svm0.fit(scaled_train_X, y_train0)
    y_pred0 = svm0.predict(scaled_train_X)
    acc0 = np.mean(y_pred0 == y_train0)
    y_val_pred0 = svm0.predict(scaled_val_X)
    val_acc0 = np.mean(y_val_pred0==y_val0)
    print(f"(lr:{lr}, C:{C_} label:0) acc: {round(acc0,4)} val acc:{round(val_acc0,3)}")
    if val_acc0>best_acc_0:
      best_acc_0 = val_acc0
      best_param0['lr'] = lr
      best_param0['C']=C_

(lr:0.0014, C:0.01 label:0) acc: 0.8225 val acc:0.831
(lr:0.0015, C:0.01 label:0) acc: 0.7444 val acc:0.752
(lr:0.0016, C:0.01 label:0) acc: 0.9101 val acc:0.915
(lr:0.0014, C:0.011 label:0) acc: 0.9103 val acc:0.915
(lr:0.0015, C:0.011 label:0) acc: 0.9191 val acc:0.925
(lr:0.0016, C:0.011 label:0) acc: 0.9223 val acc:0.93
(lr:0.0014, C:0.015 label:0) acc: 0.9061 val acc:0.911
(lr:0.0015, C:0.015 label:0) acc: 0.9191 val acc:0.925
(lr:0.0016, C:0.015 label:0) acc: 0.9027 val acc:0.908
(lr:0.0014, C:0.0112 label:0) acc: 0.9075 val acc:0.912
(lr:0.0015, C:0.0112 label:0) acc: 0.9193 val acc:0.926
(lr:0.0016, C:0.0112 label:0) acc: 0.9085 val acc:0.912


In [ ]:
print(best_param0)
print(best_acc_0)

{'lr': 0.0016, 'C': 0.011}
0.9296164491894029


In [ ]:
import copy
lrs = [0.0015,0.00155,0.0016,0.00165]
C = [0.01,0.0105,0.011,0.0115]

for C_ in C:
  for lr in lrs:
    svm0 = BinarySVM(C=C_,learning_rate=lr)
    svm0.fit(scaled_train_X, y_train0)
    y_pred0 = svm0.predict(scaled_train_X)
    acc0 = np.mean(y_pred0 == y_train0)
    y_val_pred0 = svm0.predict(scaled_val_X)
    val_acc0 = np.mean(y_val_pred0==y_val0)
    print(f"(lr:{lr}, C:{C_} label:0) acc: {round(acc0,4)} val acc:{round(val_acc0,3)}")
    if val_acc0>best_acc_0:
      best_acc_0 = val_acc0
      best_param0['lr'] = lr
      best_param0['C']=C_

(lr:0.0015, C:0.01 label:0) acc: 0.7444 val acc:0.752
(lr:0.00155, C:0.01 label:0) acc: 0.9203 val acc:0.927
(lr:0.0016, C:0.01 label:0) acc: 0.9101 val acc:0.915
(lr:0.00165, C:0.01 label:0) acc: 0.9144 val acc:0.922
(lr:0.0015, C:0.0105 label:0) acc: 0.9028 val acc:0.909
(lr:0.00155, C:0.0105 label:0) acc: 0.9027 val acc:0.908
(lr:0.0016, C:0.0105 label:0) acc: 0.9223 val acc:0.929
(lr:0.00165, C:0.0105 label:0) acc: 0.8769 val acc:0.885
(lr:0.0015, C:0.011 label:0) acc: 0.9191 val acc:0.925
(lr:0.00155, C:0.011 label:0) acc: 0.9027 val acc:0.908
(lr:0.0016, C:0.011 label:0) acc: 0.9223 val acc:0.93
(lr:0.00165, C:0.011 label:0) acc: 0.9066 val acc:0.911
(lr:0.0015, C:0.0115 label:0) acc: 0.9173 val acc:0.924
(lr:0.00155, C:0.0115 label:0) acc: 0.8563 val acc:0.866
(lr:0.0016, C:0.0115 label:0) acc: 0.9048 val acc:0.909
(lr:0.00165, C:0.0115 label:0) acc: 0.927 val acc:0.935


In [ ]:
print(best_param0)
print(best_acc_0)

{'lr': 0.00165, 'C': 0.0115}
0.9347568208778173


In [ ]:
lrs = [0.00161,0.00163,0.00164,0.00165,0.00167,0.00169]
C = [0.0114,0.0115,0.116]

for C_ in C:
  for lr in lrs:
    svm0 = BinarySVM(C=C_,learning_rate=lr,num_iterations = 5000)
    svm0.fit(scaled_train_X, y_train0)
    y_pred0 = svm0.predict(scaled_train_X)
    acc0 = np.mean(y_pred0 == y_train0)
    y_val_pred0 = svm0.predict(scaled_val_X)
    val_acc0 = np.mean(y_val_pred0==y_val0)
    print(f"(lr:{lr}, C:{C_} label:0) acc: {round(acc0,4)} val acc:{round(val_acc0,3)}")
    if val_acc0>best_acc_0 or val_acc0>0.935:
      print(">>model updated\n")
      best_acc_0 = val_acc0
      best_param0['lr'] = lr
      best_param0['C']=C_
      best_model = copy.deepcopy(svm0)

(lr:0.00161, C:0.0114 label:0) acc: 0.6646 val acc:0.655
(lr:0.00163, C:0.0114 label:0) acc: 0.9263 val acc:0.936
>>model updated

(lr:0.00164, C:0.0114 label:0) acc: 0.9134 val acc:0.919
(lr:0.00165, C:0.0114 label:0) acc: 0.9215 val acc:0.928
(lr:0.00167, C:0.0114 label:0) acc: 0.9127 val acc:0.917
(lr:0.00169, C:0.0114 label:0) acc: 0.9214 val acc:0.928
(lr:0.00161, C:0.0115 label:0) acc: 0.9135 val acc:0.919
(lr:0.00163, C:0.0115 label:0) acc: 0.9158 val acc:0.923
(lr:0.00164, C:0.0115 label:0) acc: 0.9134 val acc:0.919
(lr:0.00165, C:0.0115 label:0) acc: 0.9222 val acc:0.928
(lr:0.00167, C:0.0115 label:0) acc: 0.9116 val acc:0.915
(lr:0.00169, C:0.0115 label:0) acc: 0.9213 val acc:0.928
(lr:0.00161, C:0.116 label:0) acc: 0.9227 val acc:0.93
(lr:0.00163, C:0.116 label:0) acc: 0.9144 val acc:0.922
(lr:0.00164, C:0.116 label:0) acc: 0.9238 val acc:0.933
(lr:0.00165, C:0.116 label:0) acc: 0.9222 val acc:0.928
(lr:0.00167, C:0.116 label:0) acc: 0.3782 val acc:0.369
(lr:0.00169, C:0.116

In [ ]:
print(best_param0)
print(best_acc_0)

{'lr': 0.00163, 'C': 0.0114}
0.9355476472914195


## Save best label0 model

In [ ]:
best_param0['model'] = best_model
best_param0['acc'] = best_acc_0
best_svm_dict[0] = best_param0

# label 1

In [ ]:
import copy
lrs = [0.1,0.01,0.005,0.001,0.0005,0.0001]
C = [0.001,0.01,0.1,1.,5.]
best_param1 = dict()

best_acc_1 = 0
for C_ in C:
  for lr in lrs:
    svm1 = BinarySVM(C=C_,learning_rate=lr)
    svm1.fit(scaled_train_X, y_train1)
    y_pred1 = svm1.predict(scaled_train_X)
    y_val_pred1 = svm1.predict(scaled_val_X)
    acc1 = np.mean(y_pred1 == y_train1)
    val_acc1 = np.mean(y_val_pred1==y_val1)
    print(f"(lr:{lr}, C:{C_} label:1) acc: {round(acc1,4)} val acc:{round(val_acc1,3)}")
    if val_acc1>best_acc_1:
      best_acc_1 = val_acc1
      best_param1['lr'] = lr
      best_param1['C']=C_
      best_svm1 = copy.deepcopy(svm1)

(lr:0.1, C:0.001 label:1) acc: 0.8773 val acc:0.875
(lr:0.01, C:0.001 label:1) acc: 0.8754 val acc:0.873
(lr:0.005, C:0.001 label:1) acc: 0.902 val acc:0.905
(lr:0.001, C:0.001 label:1) acc: 0.8761 val acc:0.878
(lr:0.0005, C:0.001 label:1) acc: 0.8772 val acc:0.879
(lr:0.0001, C:0.001 label:1) acc: 0.8738 val acc:0.875
(lr:0.1, C:0.01 label:1) acc: 0.8754 val acc:0.873
(lr:0.01, C:0.01 label:1) acc: 0.8757 val acc:0.873
(lr:0.005, C:0.01 label:1) acc: 0.8756 val acc:0.873
(lr:0.001, C:0.01 label:1) acc: 0.8758 val acc:0.877
(lr:0.0005, C:0.01 label:1) acc: 0.8772 val acc:0.879
(lr:0.0001, C:0.01 label:1) acc: 0.8731 val acc:0.874
(lr:0.1, C:0.1 label:1) acc: 0.8755 val acc:0.873
(lr:0.01, C:0.1 label:1) acc: 0.8754 val acc:0.873
(lr:0.005, C:0.1 label:1) acc: 0.8755 val acc:0.873
(lr:0.001, C:0.1 label:1) acc: 0.876 val acc:0.877
(lr:0.0005, C:0.1 label:1) acc: 0.8772 val acc:0.879
(lr:0.0001, C:0.1 label:1) acc: 0.8736 val acc:0.874
(lr:0.1, C:1.0 label:1) acc: 0.9165 val acc:0.919
(

In [ ]:
print(best_param1)
print(best_acc_1)

{'lr': 0.1, 'C': 1.0}
0.9185448793989719


In [ ]:
# import copy
lrs = [0.1,0.09,0.15,0.2]
C = [0.95,1.0,1.05]

for C_ in C:
  for lr in lrs:
    svm1 = BinarySVM(C=C_,learning_rate=lr,num_iterations=305)
    svm1.fit(scaled_train_X, y_train1)
    y_pred1 = svm1.predict(scaled_train_X)
    y_val_pred1 = svm1.predict(scaled_val_X)
    acc1 = np.mean(y_pred1 == y_train1)
    val_acc1 = np.mean(y_val_pred1==y_val1)
    print(f"(lr:{lr}, C:{C_} label:1) acc: {round(acc1,4)} val acc:{round(val_acc1,3)}")
    if val_acc1>best_acc_1:
      print(">>best model updated\n")
      best_acc_1 = val_acc1
      best_param1['lr'] = lr
      best_param1['C']=C_
      best_svm1 = copy.deepcopy(svm1)

(lr:0.1, C:0.95 label:1) acc: 0.8735 val acc:0.873
(lr:0.09, C:0.95 label:1) acc: 0.8733 val acc:0.874
(lr:0.15, C:0.95 label:1) acc: 0.8827 val acc:0.886
(lr:0.2, C:0.95 label:1) acc: 0.8852 val acc:0.89
(lr:0.1, C:1.0 label:1) acc: 0.1249 val acc:0.128
(lr:0.09, C:1.0 label:1) acc: 0.8737 val acc:0.874
(lr:0.15, C:1.0 label:1) acc: 0.8735 val acc:0.874
(lr:0.2, C:1.0 label:1) acc: 0.8826 val acc:0.886
(lr:0.1, C:1.05 label:1) acc: 0.9026 val acc:0.905
(lr:0.09, C:1.05 label:1) acc: 0.876 val acc:0.877
(lr:0.15, C:1.05 label:1) acc: 0.8737 val acc:0.874
(lr:0.2, C:1.05 label:1) acc: 0.877 val acc:0.879


In [ ]:
print(best_param1)
print(best_acc_1)

{'lr': 0.15, 'C': 0.95}
0.9288256227758007


## Save best label1 model

In [ ]:
best_param1['model'] = best_svm1
best_param1['acc'] = best_acc_1
best_svm_dict[1] = best_param1

In [ ]:
best_svm_dict.keys()

dict_keys([0, 1])

# label 2

In [ ]:
import copy
lrs = [0.1,0.01,0.005,0.001,0.0005,0.0001]
C = [0.001,0.01,0.1,1.,5.]
best_param2 = dict()
best_acc_2 = 0
for C_ in C:
  for lr in lrs:
    svm2 = BinarySVM(C=C_,learning_rate=lr)
    svm2.fit(scaled_train_X, y_train2)
    y_pred2 = svm2.predict(scaled_train_X)
    y_val_pred2 = svm2.predict(scaled_val_X)
    acc2 = np.mean(y_pred2 == y_train2)
    val_acc2 = np.mean(y_val_pred2==y_val2)
    print(f"(lr:{lr}, C:{C_} label:2) acc: {round(acc2,4)} val acc:{round(val_acc2,3)}")
    if val_acc2>best_acc_2:
      print(">>best model updated\n")
      best_acc_2 = val_acc2
      best_param2['lr'] = lr
      best_param2['C']=C_
      best_svm2 = copy.deepcopy(svm2)

(lr:0.1, C:0.001 label:2) acc: 0.9285 val acc:0.925
>>best model updated

(lr:0.01, C:0.001 label:2) acc: 0.9288 val acc:0.926
>>best model updated

(lr:0.005, C:0.001 label:2) acc: 0.9288 val acc:0.926
(lr:0.001, C:0.001 label:2) acc: 0.9288 val acc:0.926
(lr:0.0005, C:0.001 label:2) acc: 0.9274 val acc:0.925
(lr:0.0001, C:0.001 label:2) acc: 0.9288 val acc:0.926
(lr:0.1, C:0.01 label:2) acc: 0.9288 val acc:0.926
(lr:0.01, C:0.01 label:2) acc: 0.9288 val acc:0.926
(lr:0.005, C:0.01 label:2) acc: 0.9288 val acc:0.926
>>best model updated

(lr:0.001, C:0.01 label:2) acc: 0.9288 val acc:0.926
(lr:0.0005, C:0.01 label:2) acc: 0.9288 val acc:0.926
(lr:0.0001, C:0.01 label:2) acc: 0.9288 val acc:0.926
(lr:0.1, C:0.1 label:2) acc: 0.9288 val acc:0.926
(lr:0.01, C:0.1 label:2) acc: 0.9288 val acc:0.926
(lr:0.005, C:0.1 label:2) acc: 0.9288 val acc:0.926
(lr:0.001, C:0.1 label:2) acc: 0.9288 val acc:0.926
(lr:0.0005, C:0.1 label:2) acc: 0.9287 val acc:0.925
(lr:0.0001, C:0.1 label:2) acc: 0.92

In [ ]:
print(best_param2)
print(best_acc_2)

{'lr': 0.005, 'C': 0.01}
0.9260577303281929


In [ ]:
lrs = [0.003,0.004,0.005,0.006,0.007]
C = [0.009,0.01,0.05,0.002]
num_iter = 300
# best_param2 = dict()
# best_acc_2 = 0
for C_ in C:
  for lr in lrs:
    svm2 = BinarySVM(C=C_,learning_rate=lr)
    svm2.fit(scaled_train_X, y_train2)
    y_pred2 = svm2.predict(scaled_train_X)
    y_val_pred2 = svm2.predict(scaled_val_X)
    acc2 = np.mean(y_pred2 == y_train2)
    val_acc2 = np.mean(y_val_pred2==y_val2)
    print(f"(lr:{lr}, C:{C_} label:2) acc: {round(acc2,4)} val acc:{round(val_acc2,3)}")
    if val_acc2>best_acc_2:
      print(">>best model updated\n")
      best_acc_2 = val_acc2
      best_param2['lr'] = lr
      best_param2['C']=C_
      best_svm2 = copy.deepcopy(svm2)

(lr:0.003, C:0.009 label:2) acc: 0.9288 val acc:0.926
(lr:0.004, C:0.009 label:2) acc: 0.9288 val acc:0.926
(lr:0.005, C:0.009 label:2) acc: 0.831 val acc:0.826
(lr:0.006, C:0.009 label:2) acc: 0.9288 val acc:0.926
(lr:0.007, C:0.009 label:2) acc: 0.2406 val acc:0.237
(lr:0.003, C:0.01 label:2) acc: 0.9288 val acc:0.926
(lr:0.004, C:0.01 label:2) acc: 0.8719 val acc:0.879
(lr:0.005, C:0.01 label:2) acc: 0.9288 val acc:0.926
(lr:0.006, C:0.01 label:2) acc: 0.9288 val acc:0.926
(lr:0.007, C:0.01 label:2) acc: 0.8663 val acc:0.873
(lr:0.003, C:0.05 label:2) acc: 0.9288 val acc:0.926
(lr:0.004, C:0.05 label:2) acc: 0.9288 val acc:0.926
(lr:0.005, C:0.05 label:2) acc: 0.9288 val acc:0.926
(lr:0.006, C:0.05 label:2) acc: 0.2266 val acc:0.219
(lr:0.007, C:0.05 label:2) acc: 0.9198 val acc:0.92
(lr:0.003, C:0.002 label:2) acc: 0.8422 val acc:0.84
(lr:0.004, C:0.002 label:2) acc: 0.9282 val acc:0.925
(lr:0.005, C:0.002 label:2) acc: 0.8118 val acc:0.814
(lr:0.006, C:0.002 label:2) acc: 0.8401 v

In [ ]:
iters = [100,200,300,400,500,1500,2000,30000]
for n_iter in iters:
  svm2 = BinarySVM(C=best_param2['C'],learning_rate=best_param2['lr'],num_iterations=n_iter)
  svm2.fit(scaled_train_X, y_train2)
  y_pred2 = svm2.predict(scaled_train_X)
  y_val_pred2 = svm2.predict(scaled_val_X)
  acc2 = np.mean(y_pred2 == y_train2)
  val_acc2 = np.mean(y_val_pred2==y_val2)
  print(f"n_iter: {n_iter} (lr:{lr}, C:{C_} label:2) acc: {round(acc2,4)} val acc:{round(val_acc2,3)}")
  if val_acc2>best_acc_2:
    print(">>best model updated\n")
    best_acc_2 = val_acc2
    best_param2['lr'] = lr
    best_param2['C']=C_
    best_param2['n_iter'] = n_iter
    best_svm2 = copy.deepcopy(svm2)

n_iter: 100 (lr:0.007, C:0.002 label:2) acc: 0.9288 val acc:0.926
n_iter: 200 (lr:0.007, C:0.002 label:2) acc: 0.9117 val acc:0.916
n_iter: 300 (lr:0.007, C:0.002 label:2) acc: 0.9288 val acc:0.926
n_iter: 400 (lr:0.007, C:0.002 label:2) acc: 0.9288 val acc:0.926
n_iter: 500 (lr:0.007, C:0.002 label:2) acc: 0.929 val acc:0.926
n_iter: 1500 (lr:0.007, C:0.002 label:2) acc: 0.9288 val acc:0.926
n_iter: 2000 (lr:0.007, C:0.002 label:2) acc: 0.9288 val acc:0.926
n_iter: 30000 (lr:0.007, C:0.002 label:2) acc: 0.9288 val acc:0.926


## Save best label2 model

In [ ]:
best_param2['model'] = best_svm2
best_param2['acc'] = best_acc_2
best_svm_dict[2] = best_param2

In [ ]:
best_svm_dict.keys()

dict_keys([0, 1, 2])

# label 3

In [ ]:
import copy
lrs = [0.1,0.01,0.005,0.001,0.0005,0.0001]
C = [0.001,0.01,0.1,1.,5.]
best_param3 = dict()
best_acc_3 = 0
for C_ in C:
  for lr in lrs:
    svm3 = BinarySVM(C=C_,learning_rate=lr)
    svm3.fit(scaled_train_X, y_train3)
    y_pred3 = svm3.predict(scaled_train_X)
    y_val_pred3 = svm3.predict(scaled_val_X)
    acc3 = np.mean(y_pred3 == y_train3)
    val_acc3 = np.mean(y_val_pred3==y_val3)
    print(f"(lr:{lr}, C:{C_} label:3) acc: {round(acc3,4)} val acc:{round(val_acc3,3)}")
    if val_acc3>best_acc_3:
      print(">>best model updated\n")
      best_acc_3 = val_acc3
      best_param3['lr'] = lr
      best_param3['C']=C_
      best_svm3 = copy.deepcopy(svm3)

(lr:0.1, C:0.001 label:3) acc: 0.9272 val acc:0.932
>>best model updated

(lr:0.01, C:0.001 label:3) acc: 0.9272 val acc:0.932
(lr:0.005, C:0.001 label:3) acc: 0.3164 val acc:0.302
(lr:0.001, C:0.001 label:3) acc: 0.9272 val acc:0.932
(lr:0.0005, C:0.001 label:3) acc: 0.9272 val acc:0.932
(lr:0.0001, C:0.001 label:3) acc: 0.9272 val acc:0.932
(lr:0.1, C:0.01 label:3) acc: 0.9272 val acc:0.932
(lr:0.01, C:0.01 label:3) acc: 0.9272 val acc:0.932
(lr:0.005, C:0.01 label:3) acc: 0.9272 val acc:0.932
(lr:0.001, C:0.01 label:3) acc: 0.9272 val acc:0.932
(lr:0.0005, C:0.01 label:3) acc: 0.9272 val acc:0.932
(lr:0.0001, C:0.01 label:3) acc: 0.9272 val acc:0.932
(lr:0.1, C:0.1 label:3) acc: 0.9272 val acc:0.932
(lr:0.01, C:0.1 label:3) acc: 0.9272 val acc:0.932
(lr:0.005, C:0.1 label:3) acc: 0.9272 val acc:0.932
(lr:0.001, C:0.1 label:3) acc: 0.9272 val acc:0.932
(lr:0.0005, C:0.1 label:3) acc: 0.9272 val acc:0.932
(lr:0.0001, C:0.1 label:3) acc: 0.9272 val acc:0.932
(lr:0.1, C:1.0 label:3) acc

In [ ]:
print(best_param3)
print(best_acc_3)

{'lr': 0.1, 'C': 0.001}
0.9319889284302095


In [ ]:
import copy
lrs = [0.8,0.9,0.95,0.1,0.15]
C = [0.0001,0.0005,0.001,0.003,0.005]
# best_param3 = dict()
# best_acc_3 = 0
for C_ in C:
  for lr in lrs:
    svm3 = BinarySVM(C=C_,learning_rate=lr)
    svm3.fit(scaled_train_X, y_train3)
    y_pred3 = svm3.predict(scaled_train_X)
    y_val_pred3 = svm3.predict(scaled_val_X)
    acc3 = np.mean(y_pred3 == y_train3)
    val_acc3 = np.mean(y_val_pred3==y_val3)
    print(f"(lr:{lr}, C:{C_} label:3) acc: {round(acc3,4)} val acc:{round(val_acc3,3)}")
    if val_acc3>best_acc_3:
      print(">>best model updated\n")
      best_acc_3 = val_acc3
      best_param3['lr'] = lr
      best_param3['C']=C_
      best_svm3 = copy.deepcopy(svm3)

(lr:0.8, C:0.0001 label:3) acc: 0.9272 val acc:0.932
(lr:0.9, C:0.0001 label:3) acc: 0.9272 val acc:0.932
(lr:0.95, C:0.0001 label:3) acc: 0.9272 val acc:0.932
(lr:0.1, C:0.0001 label:3) acc: 0.9272 val acc:0.932
(lr:0.15, C:0.0001 label:3) acc: 0.9272 val acc:0.932
(lr:0.8, C:0.0005 label:3) acc: 0.9197 val acc:0.928
(lr:0.9, C:0.0005 label:3) acc: 0.9272 val acc:0.932
(lr:0.95, C:0.0005 label:3) acc: 0.9272 val acc:0.932
(lr:0.1, C:0.0005 label:3) acc: 0.9272 val acc:0.932
(lr:0.15, C:0.0005 label:3) acc: 0.9272 val acc:0.932
(lr:0.8, C:0.001 label:3) acc: 0.902 val acc:0.914
(lr:0.9, C:0.001 label:3) acc: 0.9272 val acc:0.932
(lr:0.95, C:0.001 label:3) acc: 0.9272 val acc:0.932
(lr:0.1, C:0.001 label:3) acc: 0.9272 val acc:0.932
(lr:0.15, C:0.001 label:3) acc: 0.9272 val acc:0.932
(lr:0.8, C:0.003 label:3) acc: 0.9272 val acc:0.932
(lr:0.9, C:0.003 label:3) acc: 0.9272 val acc:0.932
(lr:0.95, C:0.003 label:3) acc: 0.9147 val acc:0.922
(lr:0.1, C:0.003 label:3) acc: 0.9272 val acc:0.

In [ ]:
iters = [100,200,300,400,500,1500,2000,30000]
for n_iter in iters:
  svm3 = BinarySVM(C=best_param3['C'],learning_rate=best_param3['lr'],num_iterations=n_iter)
  svm3.fit(scaled_train_X, y_train3)
  y_pred3 = svm3.predict(scaled_train_X)
  y_val_pred3 = svm3.predict(scaled_val_X)
  acc3 = np.mean(y_pred3 == y_train3)
  val_acc3 = np.mean(y_val_pred3==y_val3)
  print(f"n_iter: {n_iter} (lr:{lr}, C:{C_} label:3) acc: {round(acc3,4)} val acc:{round(val_acc3,3)}")
  if val_acc3>best_acc_3:
    print(">>best model updated\n")
    best_acc_3 = val_acc3
    best_param3['lr'] = lr
    best_param3['C']=C_
    best_param3['n_iter'] = n_iter
    best_svm3 = copy.deepcopy(svm3)

n_iter: 100 (lr:0.15, C:0.005 label:3) acc: 0.9268 val acc:0.932
n_iter: 200 (lr:0.15, C:0.005 label:3) acc: 0.9272 val acc:0.932
n_iter: 300 (lr:0.15, C:0.005 label:3) acc: 0.9272 val acc:0.932
n_iter: 400 (lr:0.15, C:0.005 label:3) acc: 0.9272 val acc:0.932
n_iter: 500 (lr:0.15, C:0.005 label:3) acc: 0.9272 val acc:0.932
n_iter: 1500 (lr:0.15, C:0.005 label:3) acc: 0.9272 val acc:0.932
n_iter: 2000 (lr:0.15, C:0.005 label:3) acc: 0.9272 val acc:0.932
n_iter: 30000 (lr:0.15, C:0.005 label:3) acc: 0.9272 val acc:0.932


In [ ]:
Counter(y_val_pred3)

Counter({0: 2529})

In [ ]:
best_param3['model'] = best_svm3
best_param3['acc'] = best_acc_3
best_svm_dict[3] = best_param3

In [ ]:
best_svm_dict.keys()

dict_keys([0, 1, 2, 3])

# Predict on merged data

## Train (0.84)

In [ ]:
Counter(y_train)

Counter({0: 6290, 1: 2370, 3: 736, 2: 720})

In [ ]:
train_preds = np.zeros((scaled_train_X.shape[0],4))

train_pred_prob0 = best_svm_dict[0]['model'].predict_proba(scaled_train_X)
train_preds[:,0] = train_pred_prob0[:,1]

train_pred_prob1 = best_svm_dict[1]['model'].predict_proba(scaled_train_X)
train_preds[:,1] = train_pred_prob1[:,1]

train_pred_prob2 = best_svm_dict[2]['model'].predict_proba(scaled_train_X)
train_preds[:,2] = train_pred_prob2[:,1]

train_pred_prob3 = best_svm_dict[3]['model'].predict_proba(scaled_train_X)
train_preds[:,3] = 10*train_pred_prob3[:,1]

<ipython-input-64-324115d43340>:45: RuntimeWarning: overflow encountered in exp
  probs = 1 / (1 + np.exp(-scores))


In [ ]:
one_vs_all_preds = np.argmax(train_preds,axis=1)
train_acc = np.mean(one_vs_all_preds == y_train)
train_acc

0.8431198102016607

## Val (0.84)

In [ ]:
val_preds = np.zeros((scaled_val_X.shape[0],4))

val_pred_prob0 = best_svm_dict[0]['model'].predict_proba(scaled_val_X)
val_preds[:,0] = val_pred_prob0[:,1]

val_pred_prob1 = best_svm_dict[1]['model'].predict_proba(scaled_val_X)
val_preds[:,1] = val_pred_prob1[:,1]

val_pred_prob2 = best_svm_dict[2]['model'].predict_proba(scaled_val_X)
val_preds[:,2] = val_pred_prob2[:,1]

val_pred_prob3 = best_svm_dict[3]['model'].predict_proba(scaled_val_X)
val_preds[:,3] = val_pred_prob3[:,1]

<ipython-input-64-324115d43340>:45: RuntimeWarning: overflow encountered in exp
  probs = 1 / (1 + np.exp(-scores))


In [ ]:
one_vs_all_preds = np.argmax(val_preds,axis=1)
val_acc = np.mean(one_vs_all_preds == y_val)
val_acc

0.8465796757611704

In [ ]:
import torch
torch.save(best_svm_dict,"./one_vs_all_w.pt")

# Naive Bayes

In [ ]:
import numpy as np

class NaiveBayes:
    def __init__(self):
        self.priors = None  # class priors
        self.means = None  # class means
        self.variances = None  # class variances
        self.classes = None  # unique class labels

    def fit(self, X, y):
        # compute class priors
        self.classes, counts = np.unique(y, return_counts=True)
        self.priors = counts / len(y)

        # compute class means and variances
        self.means = np.zeros((len(self.classes), X.shape[1]))
        self.variances = np.zeros((len(self.classes), X.shape[1]))
        for i, c in enumerate(self.classes):
            X_c = X[y == c]
            self.means[i] = np.mean(X_c, axis=0)
            self.variances[i] = np.var(X_c, axis=0)

    def predict(self, X):
        # compute log probabilities for each class
        log_probs = np.zeros((len(X), len(self.classes)))
        for i, c in enumerate(self.classes):
            log_probs[:, i] = np.sum(-0.5 * np.log(2 * np.pi * self.variances[i])
                                     - 0.5 * ((X - self.means[i]) ** 2) / self.variances[i], axis=1)
        # add log priors and compute class probabilities
        log_probs += np.log(self.priors)
        probs = np.exp(log_probs)
        # normalize probabilities and return class with highest probability
        probs /= np.sum(probs, axis=1, keepdims=True)
        return np.argmax(probs, axis=1)


In [ ]:

X = scaled_train_X
y = y_train

# train Naive Bayes classifier
nb = NaiveBayes()
nb.fit(X, y)

# predict labels
y_pred = nb.predict(X)

# compute accuracy
acc = np.mean(y_pred == y_train)
print('Accuracy:', acc)

Accuracy: 0.799624357453539
